## LIBRERIAS

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import pyodbc
from fake_useragent import UserAgent
import random
from urllib.parse import unquote, urlparse, parse_qs
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from queue import Queue

### 1. Configuración Inicial
Definición de User-Agents, parametros de conección a la base de datos, configuración para el multithreading

In [9]:

# Configuración avanzada
ua = UserAgent(
    browsers=['chrome', 'firefox', 'safari'],  # Navegadores comunes
    platforms=['windows', 'macos', 'linux'],   # Sistemas operativos
    min_percentage=1.5  # Solo agentes con al menos 1.5% de uso global
)


# Configuración de SQL Server
SERVER = 'TuServidor'  # Ejemplo: 'LAPTOP-ABC123\SQLEXPRESS' o 'localhost'
DATABASE = 'TuBaseDeDatos'  # Nombre de tu base de datos
USERNAME = 'TuUsername'     # Dejar en blanco si usas autenticación de Windows
PASSWORD = 'TuPassword'  # Dejar en blanco si usas autenticación de Windows
TRUSTED_CONNECTION = 'yes'  # Usar 'yes' para autenticación de Windows, 'no' para SQL auth

# Configuración de threading
MAX_WORKERS = 4  # Número de hilos concurrentes (ajustar según tu conexión)
REQUEST_DELAY = (1, 2)  # Delay entre requests en segundos (min, max)

# Lock para thread-safe operations
print_lock = threading.Lock()
stats_lock = threading.Lock()

# Estadísticas globales
global_stats = {
    'pages_processed': 0,
    'products_found': 0,
    'errors': 0
}

### 2. Funciones para la extracción de datos

In [3]:
def thread_safe_print(message):
    """Print thread-safe con lock"""
    with print_lock:
        print(message)

def update_stats(pages=0, products=0, errors=0):
    """Actualiza estadísticas de forma thread-safe"""
    with stats_lock:
        global_stats['pages_processed'] += pages
        global_stats['products_found'] += products
        global_stats['errors'] += errors

def get_random_headers():
    return {
        'User-Agent': ua.random,
        'Accept-Language': 'es-ES,es;q=0.9',
        'Referer': random.choice([
            'https://www.google.com/',
            'https://www.mercadolibre.com.ar/',
            'https://www.bing.com/'
        ]),
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'DNT': str(random.randint(1, 2))
    }


def clean_url(url: str) -> str:
    """
    Limpia y acorta las URLs de MercadoLibre eliminando parámetros de tracking.
    """
    if not url:
        return ''
    
    try:
        # Si es una URL de click tracking, extraer la URL real
        if 'mclics/clicks' in url:
            # Buscar el parámetro que contiene la URL real
            parsed = urlparse(url)
            query_params = parse_qs(parsed.query)
            
            # Intentar extraer la URL real de los parámetros
            for param_name, param_values in query_params.items():
                if param_values and 'mercadolibre.com.ar' in str(param_values[0]):
                    return unquote(param_values[0])
        
        # Si no es URL de tracking, limpiar parámetros innecesarios
        parsed = urlparse(url)
        clean_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        
        # Mantener solo parámetros esenciales
        if parsed.query:
            query_params = parse_qs(parsed.query)
            essential_params = {}
            for key in ['id', 'item_id', 'category_id']:
                if key in query_params:
                    essential_params[key] = query_params[key][0]
            
            if essential_params:
                from urllib.parse import urlencode
                clean_url += '?' + urlencode(essential_params)
        
        return clean_url
    except Exception:
        # Si hay error, devolver la URL original pero truncada
        return url[:800] if len(url) > 800 else url
    

def clean_mercadolibre_search_url(url: str) -> str:
    """
    Limpia las URLs de búsqueda de MercadoLibre eliminando parámetros y fragmentos.
    
    Ejemplos:
    - Input:  https://listado.mercadolibre.com.ar/colchon-inflable?sb=all_mercadolibre#D[A:colchon%20inflable]
    - Output: https://listado.mercadolibre.com.ar/colchon-inflable
    
    - Input:  https://listado.mercadolibre.com.ar/pantalon-cargo#D[A:pantalon%20cargo]
    - Output: https://listado.mercadolibre.com.ar/pantalon-cargo
    
    Args:
        url (str): URL de MercadoLibre con parámetros y fragmentos
        
    Returns:
        str: URL limpia sin parámetros ni fragmentos
    """
    if not url:
        return ''
    
    try:
        # Parsear la URL
        parsed = urlparse(url)
        
        # Construir URL limpia: scheme + netloc + path (sin query ni fragment)
        clean_url = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        
        return clean_url
        
    except Exception as e:
        print(f"Error al limpiar URL: {e}")
        return url  # Devolver URL original si hay error


def extract_product_image(item):
    # Suponiendo que `item` es el contenedor del producto
    img_tag = item.find('img', class_='poly-component__picture')

    # 1) Revisar data-src o data-original
    img_url = None
    if img_tag:
        img_url = (
            img_tag.get('data-src') or
            img_tag.get('data-original') or
            img_tag.get('data-lazy')  # a veces usan este nombre
        )

    # 2) Si usan srcset, tomar la primera URL
    if not img_url and img_tag and img_tag.get('srcset'):
        # srcset suele ser algo como "https://...-E.webp 1x, https://...-F.webp 2x"
        srcset = img_tag['srcset']
        # Partimos por comas y luego espacio antes de la densidad
        img_url = srcset.split(',')[0].split()[0]

    # 3) Fallback a src (no recomendado, es el placeholder)
    if not img_url and img_tag:
        img_url = img_tag.get('src')

    return img_url

def scrape_single_page(page_info: dict) -> list[dict]:
    """
    Función optimizada para scraping de una sola página - thread-safe.
    Recibe un diccionario con 'url', 'page_num' y 'thread_id'
    """
    page_url = page_info['url']
    page_num = page_info['page_num']
    thread_id = page_info['thread_id']
    
    results = []
    
    try:
        # Delay aleatorio para evitar sobrecargar el servidor
        delay = random.uniform(*REQUEST_DELAY)
        time.sleep(delay)
        
        thread_safe_print(f"🔄 [Hilo {thread_id}] Procesando página {page_num}: {page_url}")
        
        # Realizar request con headers aleatorios
        session = requests.Session()
        response = session.get(page_url, headers=get_random_headers(), timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        items = soup.find_all('li', class_='ui-search-layout__item')
        
        if not items:
            thread_safe_print(f"⚠️  [Hilo {thread_id}] No se encontraron productos en página {page_num}")
            update_stats(pages=1, errors=1)
            return results
        
        for item in items:
            try:
                # Nombre del producto
                name_element = item.find('h3', class_='poly-component__title-wrapper')
                name = name_element.text.strip() if name_element else 'Sin nombre'

                # Precio del producto
                price_element = item.find('span', class_='andes-money-amount andes-money-amount--cents-superscript')
                if price_element:
                    price_text = price_element.text.replace('$', '').replace('.', '').replace(',', '')
                    price_clean = int(re.sub(r'[^\d]', '', price_text)) if price_text else 0
                else:
                    price_clean = 0
                
                # Link
                link_element = item.find("a", class_='poly-component__title', href=True)
                link = link_element["href"] if link_element else ''
                
                # Ventas
                try:
                    sales_element = item.find('span', class_='poly-reviews__total')
                    sales = sales_element.text.strip().replace('(', '').replace(')', '') if sales_element else '0'
                    sales_clean = int(re.sub(r'[^\d]', '', sales)) if sales else 0
                except (AttributeError, ValueError):
                    sales_clean = 0
                    
                # Calificación
                try:
                    rate_element = item.find('span', class_='poly-reviews__rating')
                    rate = rate_element.text.strip() if rate_element else '0'
                    rate_clean = float(rate.replace(',', '.')) if rate else 0.0
                except (AttributeError, ValueError):
                    rate_clean = 0.0
                
                results.append({
                    'url': clean_url(link),
                    'nombre': name,
                    'precio': price_clean,
                    'ventas': sales_clean,
                    'calificacion': rate_clean,
                    'imagen': extract_product_image(item)
                })
                
            except Exception as e:
                thread_safe_print(f"❌ [Hilo {thread_id}] Error procesando producto: {e}")
                continue
        
        # Actualizar estadísticas
        update_stats(pages=1, products=len(results))
        thread_safe_print(f"✅ [Hilo {thread_id}] Página {page_num} completada: {len(results)} productos")
        
        session.close()
        
    except requests.exceptions.RequestException as e:
        thread_safe_print(f"❌ [Hilo {thread_id}] Error de red en página {page_num}: {e}")
        update_stats(pages=1, errors=1)
    except Exception as e:
        thread_safe_print(f"❌ [Hilo {thread_id}] Error general en página {page_num}: {e}")
        update_stats(pages=1, errors=1)
    
    return results

def get_all_products_threaded(base_url: str, max_pages: int = 10, max_workers: int = MAX_WORKERS) -> list[dict]:
    """
    Version con multihilos para extraer productos de múltiples páginas concurrentemente.
    CORREGIDA para manejar correctamente la paginación de MercadoLibre.
    """
    thread_safe_print(f"🚀 Iniciando scraping multihilo con {max_workers} workers")
    
    # URL base para páginas 2 en adelante (diferente a la primera página)
    #PAGINATED_BASE_URL = "https://listado.mercadolibre.com.ar/campera-polar"
    url_clean=clean_mercadolibre_search_url(base_url)
    
    # Preparar lista de páginas para procesar
    page_tasks = []
    for i in range(max_pages):
        if i == 0:
            # Primera página: usar la URL original
            page_url = base_url
        else:
            # Páginas 2 en adelante: usar la nueva estructura
            # Offset: página 2 = 49 (1 + 48*1), página 3 = 97 (1 + 48*2), etc.
            offset = 1 + (48 * i)
            page_url = f"{url_clean}_Desde_{offset}_NoIndex_True"
        
        page_tasks.append({
            'url': page_url,
            'page_num': i + 1,
            'thread_id': f"T{(i % max_workers) + 1}"
        })
        
        # Debug: mostrar las URLs generadas
        thread_safe_print(f"📄 Página {i + 1}: {page_url}")
    
    all_products = []
    
    # Ejecutar scraping con ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enviar todas las tareas
        future_to_page = {
            executor.submit(scrape_single_page, page_task): page_task 
            for page_task in page_tasks
        }
        
        # Procesar resultados conforme se completan
        for future in as_completed(future_to_page):
            page_task = future_to_page[future]
            try:
                page_results = future.result()
                all_products.extend(page_results)
                
                # Si una página no tiene productos, podría ser el final
                if not page_results:
                    thread_safe_print(f"⚠️  Página {page_task['page_num']} sin productos")
                    
            except Exception as e:
                thread_safe_print(f"❌ Error procesando página {page_task['page_num']}: {e}")
    
    thread_safe_print(f"\n📊 ESTADÍSTICAS FINALES:")
    thread_safe_print(f"   • Páginas procesadas: {global_stats['pages_processed']}")
    thread_safe_print(f"   • Productos encontrados: {global_stats['products_found']}")
    thread_safe_print(f"   • Errores: {global_stats['errors']}")
    thread_safe_print(f"   • Total productos únicos: {len(all_products)}")
    
    return all_products

### funcion para la conección a la base de datos y main()

In [4]:
def create_db_connection():
    """Establece conexión con SQL Server"""
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};'
    
    # Determinar tipo de autenticación
    if TRUSTED_CONNECTION == 'yes':
        conn_str += 'Trusted_Connection=yes;'
    else:
        conn_str += f'UID={USERNAME};PWD={PASSWORD};'
    
    try:
        conn = pyodbc.connect(conn_str)
        thread_safe_print("Conexión a la base de datos establecida correctamente.")
        return conn
    except Exception as e:
        thread_safe_print(f"Error de conexión a la base de datos: {e}")
        return None

def create_products_table(conn):
    """Crea la tabla de productos si no existe"""
    cursor = conn.cursor()
    try:
        # Verificar si la tabla existe
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'ProductosMercadoLibre')
        BEGIN
            CREATE TABLE ProductosMercadoLibre (
                ID INT IDENTITY(1,1) PRIMARY KEY,
                URL NVARCHAR(MAX),
                Nombre NVARCHAR(1000),
                Precio INT,
                Calificacion FLOAT,
                Ventas INT,
                ImagenURL NVARCHAR(MAX),
                FechaExtraccion DATETIME DEFAULT GETDATE()
            )
        END
        ELSE
        BEGIN
            -- Si la tabla existe, verificar y actualizar las columnas si es necesario
            IF EXISTS (SELECT * FROM sys.columns WHERE object_id = OBJECT_ID('ProductosMercadoLibre') AND name = 'URL' AND max_length < 0)
            BEGIN
                PRINT 'Columnas ya tienen el tamaño correcto'
            END
            ELSE
            BEGIN
                ALTER TABLE ProductosMercadoLibre ALTER COLUMN URL NVARCHAR(MAX)
                ALTER TABLE ProductosMercadoLibre ALTER COLUMN Nombre NVARCHAR(1000)
                ALTER TABLE ProductosMercadoLibre ALTER COLUMN ImagenURL NVARCHAR(MAX)
                PRINT 'Columnas actualizadas para soportar URLs más largas'
            END
        END
        ''')
        conn.commit()
        thread_safe_print("Tabla de productos verificada/creada correctamente.")
    except Exception as e:
        thread_safe_print(f"Error al crear/actualizar la tabla: {e}")
    finally:
        cursor.close()

def insert_products_to_db_batch(conn, products: list[dict], batch_size: int = 100):
    """
    Inserta los productos en la base de datos por lotes para mejor rendimiento.
    """
    if not products:
        thread_safe_print("No hay productos para insertar.")
        return
    
    total_products = len(products)
    thread_safe_print(f"💾 Insertando {total_products} productos en lotes de {batch_size}...")
    
    cursor = conn.cursor()
    try:
        # Query de inserción
        insert_query = '''
        INSERT INTO ProductosMercadoLibre 
        (URL, Nombre, Precio, Calificacion, Ventas, ImagenURL)
        VALUES (?, ?, ?, ?, ?, ?)
        '''
        
        # Procesar por lotes
        for i in range(0, total_products, batch_size):
            batch = products[i:i + batch_size]
            
            # Preparar los datos para la inserción
            insert_data = []
            for product in batch:
                insert_data.append((
                    product.get('url', ''),
                    product.get('nombre', ''),
                    product.get('precio', 0),
                    product.get('calificacion', 0.0),
                    product.get('ventas', 0),
                    product.get('imagen', '')
                ))
            
            # Inserción del lote
            cursor.executemany(insert_query, insert_data)
            conn.commit()
            
            progress = min(i + batch_size, total_products)
            thread_safe_print(f"   ✅ Lote {progress}/{total_products} productos insertados")
        
        thread_safe_print(f"🎉 {total_products} productos insertados correctamente en la base de datos.")
        
    except Exception as e:
        thread_safe_print(f"❌ Error al insertar productos: {e}")
        conn.rollback()
    finally:
        cursor.close()

def export_to_csv(products: list[dict], filename: str = 'productos_mercadolibre.csv'):
    """
    Exporta los productos a un archivo CSV como respaldo.
    """
    try:
        df = pd.DataFrame(products)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        thread_safe_print(f"📄 Datos exportados a {filename}")
    except Exception as e:
        thread_safe_print(f"Error al exportar CSV: {e}")

def remove_duplicates(products: list[dict]) -> list[dict]:
    """
    Elimina productos duplicados basándose en la URL.
    """
    seen_urls = set()
    unique_products = []
    
    for product in products:
        url = product.get('url', '')
        if url and url not in seen_urls:
            seen_urls.add(url)
            unique_products.append(product)
    
    removed_count = len(products) - len(unique_products)
    if removed_count > 0:
        thread_safe_print(f"🔄 Eliminados {removed_count} productos duplicados")
    
    return unique_products

def main():
    """
    Función principal que ejecuta todo el proceso de scraping con multihilos.
    """
    print("🚀 Iniciando scraping MULTIHILO de MercadoLibre - Componentes de PC")
    print("=" * 70)
    
    # Configuración
    URL_scraping = input("Introduce la URL de la página que quieres scrapear: ").strip()
    max_pages = int(input("¿Cuántas páginas quieres scrapear? (default: 5): ") or "5")
    max_workers = int(input(f"¿Cuántos hilos usar? (default: {MAX_WORKERS}, max recomendado: 6): ") or str(MAX_WORKERS))
    export_csv = input("¿Exportar también a CSV? (s/n, default: s): ").lower() != 'n'
    
    # Validar configuración
    max_workers = min(max_workers, 8)  # Límite máximo para ser respetuosos
    
    print(f"\n⚙️ CONFIGURACIÓN:")
    print(f"   • Páginas a procesar: {max_pages}")
    print(f"   • Hilos concurrentes: {max_workers}")
    print(f"   • Delay entre requests: {REQUEST_DELAY[0]}-{REQUEST_DELAY[1]} segundos")
    print(f"   • Exportar CSV: {'Sí' if export_csv else 'No'}")
    
    # Resetear estadísticas globales
    global_stats['pages_processed'] = 0
    global_stats['products_found'] = 0
    global_stats['errors'] = 0
    
    start_time = time.time()
    
    # 1. Extraer productos con multihilos
    print(f"\n📥 Iniciando extracción multihilo...")
    products = get_all_products_threaded(URL_scraping, max_pages, max_workers)
    
    if not products:
        print("❌ No se pudieron extraer productos. Verifica la conexión y la URL.")
        return
    
    # 2. Eliminar duplicados
    products = remove_duplicates(products)
    
    extraction_time = time.time() - start_time
    print(f"✅ Extracción completada en {extraction_time:.2f} segundos")
    print(f"   • Productos únicos obtenidos: {len(products)}")
    print(f"   • Velocidad promedio: {len(products)/extraction_time:.2f} productos/segundo")
    
    # 3. Exportar a CSV (opcional)
    if export_csv:
        export_to_csv(products)
    
    # 4. Conectar a la base de datos
    print("\n🔌 Conectando a la base de datos...")
    conn = create_db_connection()
    
    if not conn:
        print("❌ No se pudo conectar a la base de datos.")
        return
    
    # 5. Crear tabla si no existe
    create_products_table(conn)
    
    # 6. Insertar productos por lotes
    print("\n💾 Insertando productos en la base de datos...")
    insert_products_to_db_batch(conn, products, batch_size=50)
    
    # 7. Cerrar conexión
    conn.close()
    
    total_time = time.time() - start_time
    print(f"\n🎉 Proceso completado exitosamente en {total_time:.2f} segundos!")
    
    # 8. Mostrar resumen final
    print(f"\n📊 RESUMEN FINAL:")
    print(f"   • Tiempo total: {total_time:.2f} segundos")
    print(f"   • Productos extraídos: {len(products)}")
    print(f"   • Páginas procesadas: {global_stats['pages_processed']}")
    print(f"   • Hilos utilizados: {max_workers}")
    print(f"   • Eficiencia: {len(products)/total_time:.2f} productos/segundo")
    



In [10]:

if __name__ == "__main__":
    main()

🚀 Iniciando scraping MULTIHILO de MercadoLibre - Componentes de PC

⚙️ CONFIGURACIÓN:
   • Páginas a procesar: 2
   • Hilos concurrentes: 4
   • Delay entre requests: 1-2 segundos
   • Exportar CSV: No

📥 Iniciando extracción multihilo...
🚀 Iniciando scraping multihilo con 4 workers
📄 Página 1: https://listado.mercadolibre.com.ar/sansung-galaxy-ao4#D[A:sansung%20galaxy%20ao4,L:undefined]&origin=UNKNOWN&as.comp_t=SUG&as.comp_v=%0A&as.comp_id=HIS
📄 Página 2: https://listado.mercadolibre.com.ar/sansung-galaxy-ao4_Desde_49_NoIndex_True


Error occurred during getting browser(s): random, but was suppressed with fallback.


🔄 [Hilo T1] Procesando página 1: https://listado.mercadolibre.com.ar/sansung-galaxy-ao4#D[A:sansung%20galaxy%20ao4,L:undefined]&origin=UNKNOWN&as.comp_t=SUG&as.comp_v=%0A&as.comp_id=HIS


Error occurred during getting browser(s): random, but was suppressed with fallback.


🔄 [Hilo T2] Procesando página 2: https://listado.mercadolibre.com.ar/sansung-galaxy-ao4_Desde_49_NoIndex_True
✅ [Hilo T1] Página 1 completada: 50 productos
✅ [Hilo T2] Página 2 completada: 50 productos

📊 ESTADÍSTICAS FINALES:
   • Páginas procesadas: 2
   • Productos encontrados: 100
   • Errores: 0
   • Total productos únicos: 100
🔄 Eliminados 2 productos duplicados
✅ Extracción completada en 3.59 segundos
   • Productos únicos obtenidos: 98
   • Velocidad promedio: 27.31 productos/segundo

🔌 Conectando a la base de datos...
Conexión a la base de datos establecida correctamente.
Tabla de productos verificada/creada correctamente.

💾 Insertando productos en la base de datos...
💾 Insertando 98 productos en lotes de 50...
   ✅ Lote 50/98 productos insertados
   ✅ Lote 98/98 productos insertados
🎉 98 productos insertados correctamente en la base de datos.

🎉 Proceso completado exitosamente en 3.77 segundos!

📊 RESUMEN FINAL:
   • Tiempo total: 3.77 segundos
   • Productos extraídos: 98
 